In [ ]:
from calibration_functions import *

import cv2
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import sys

In [ ]:
with open('../config.json') as f:
    config = json.load(f)
BASE_DIR = config["BASE_DIR"]

# Rectification script

In [ ]:

image_list_left = [
    BASE_DIR + 'data_1/multicast/camera1/image_000.jpg',
    BASE_DIR + 'data_1/multicast/camera1/image_001.jpg',
    # BASE_DIR + 'data_1/multicast/camera1/image_002.jpg',
    # BASE_DIR + 'data_1/multicast/camera1/image_003.jpg',
    BASE_DIR + 'data_1/multicast/camera1/image_004.jpg',
    BASE_DIR + 'data_1/multicast/camera1/image_005.jpg',
    BASE_DIR + 'data_1/multicast/camera1/image_006.jpg',
    # BASE_DIR + 'data_1/multicast/camera1/image_007.jpg',
    # BASE_DIR + 'data_1/multicast/camera1/image_008.jpg',
    BASE_DIR + 'data_1/multicast/camera1/image_009.jpg',
    BASE_DIR + 'data_1/multicast/camera1/image_010.jpg',
    BASE_DIR + 'data_1/multicast/camera1/image_011.jpg',
    # BASE_DIR + 'data_1/multicast/camera1/image_012.jpg',
    BASE_DIR + 'data_1/multicast/camera1/image_013.jpg',
]

image_list_right = [
    BASE_DIR + 'data_1/multicast/camera2/image_000.jpg',
    BASE_DIR + 'data_1/multicast/camera2/image_001.jpg',
    # BASE_DIR + 'data_1/multicast/camera2/image_002.jpg',
    # BASE_DIR + 'data_1/multicast/camera2/image_003.jpg',
    BASE_DIR + 'data_1/multicast/camera2/image_004.jpg',
    BASE_DIR + 'data_1/multicast/camera2/image_005.jpg',
    BASE_DIR + 'data_1/multicast/camera2/image_006.jpg',
    # BASE_DIR + 'data_1/multicast/camera2/image_007.jpg',
    # BASE_DIR + 'data_1/multicast/camera2/image_008.jpg',
    BASE_DIR + 'data_1/multicast/camera2/image_009.jpg',
    BASE_DIR + 'data_1/multicast/camera2/image_010.jpg',
    BASE_DIR + 'data_1/multicast/camera2/image_011.jpg',
    # BASE_DIR + 'data_1/multicast/camera2/image_012.jpg',
    BASE_DIR + 'data_1/multicast/camera2/image_013.jpg',
]

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
chessboard_size = (6, 10)
frame_size = cv2.imread(image_list_left[0]).shape[:2][::-1]

objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
# objp = objp * 17.5

objpoints = []
imgpoints_left = []
imgpoints_right = []

for fname_left, fname_right in zip(image_list_left, image_list_right):
    img_left = cv2.imread(fname_left)
    img_right = cv2.imread(fname_right)
    gray_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

    ret_left, corners_left = cv2.findChessboardCorners(gray_left, chessboard_size, None)
    ret_right, corners_right = cv2.findChessboardCorners(gray_right, chessboard_size, None)
    
    if ret_left and ret_right:
        objpoints.append(objp)

        corners_left = cv2.cornerSubPix(gray_left, corners_left, (11, 11), (-1, -1), criteria)
        imgpoints_left.append(corners_left)

        corners_right = cv2.cornerSubPix(gray_right, corners_right, (11, 11), (-1, -1), criteria)
        imgpoints_right.append(corners_right)
        
        cv2.drawChessboardCorners(img_left, chessboard_size, corners_left, ret_left)
        cv2.drawChessboardCorners(img_right, chessboard_size, corners_right, ret_right)
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 10))
        axes[0].imshow(cv2.cvtColor(img_left, cv2.COLOR_BGR2RGB))
        axes[0].axis('off')
        
        axes[1].imshow(cv2.cvtColor(img_right, cv2.COLOR_BGR2RGB))
        axes[1].axis('off')
        
        plt.show()

ret_left, mtx_left, dist_left, rvecs_left, tvecs_left = cv2.calibrateCamera(objpoints, imgpoints_left, frame_size, None, None)
newcameramtx_left, roi_left = cv2.getOptimalNewCameraMatrix(mtx_left, dist_left, frame_size, 0, frame_size)

ret_right, mtx_right, dist_right, rvecs_right, tvecs_right = cv2.calibrateCamera(objpoints, imgpoints_right, frame_size, None, None)
newcameramtx_right, roi_right = cv2.getOptimalNewCameraMatrix(mtx_right, dist_right, frame_size, 0, frame_size)

flags = cv2.CALIB_FIX_INTRINSIC
ret, mtx_left, dist_left, mtx_right, dist_right, R, T, E, F = cv2.stereoCalibrate(
    objpoints, imgpoints_left, imgpoints_right, newcameramtx_left, dist_left, newcameramtx_right, dist_right, frame_size, criteria=criteria, flags=flags
)

R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(mtx_left, dist_left, mtx_right, dist_right, frame_size, R, T, 1)

map1_left, map2_left = cv2.initUndistortRectifyMap(mtx_left, dist_left, R1, P1, frame_size, cv2.CV_16SC2)
map1_right, map2_right = cv2.initUndistortRectifyMap(mtx_right, dist_right, R2, P2, frame_size, cv2.CV_16SC2)

for fname_left, fname_right in zip(image_list_left, image_list_right):
    img_left = cv2.imread(fname_left)
    img_right = cv2.imread(fname_right)
    
    img_left_rectified = cv2.remap(img_left, map1_left, map2_left, cv2.INTER_LANCZOS4)
    img_right_rectified = cv2.remap(img_right, map1_right, map2_right, cv2.INTER_LANCZOS4)

    img_left_rectified = cv2.remap(img_left, map1_left, map2_left, cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
    img_right_rectified = cv2.remap(img_right, map1_right, map2_right, cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 10))
    axes[0].imshow(cv2.cvtColor(img_left_rectified, cv2.COLOR_BGR2RGB))
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(img_right_rectified, cv2.COLOR_BGR2RGB))
    axes[1].axis('off')

    plt.show()

# Rectify dataset 1

In [ ]:
# pmf_path           = BASE_DIR + '/data_1/ground_truth_pfm_disparity_half'
image_folder_left  = os.path.join(BASE_DIR, 'data_1/multicast/camera1')
image_folder_right = os.path.join(BASE_DIR, 'data_1/multicast/camera2')


# list_of_pfm_files = sorted([pfm for pfm in os.listdir(pmf_path) if pfm.endswith('.pfm')])
list_of_image_files_left = sorted([image for image in os.listdir(image_folder_left) if image.endswith('.jpg')])
list_of_image_files_right = sorted([image for image in os.listdir(image_folder_right) if image.endswith('.jpg')])

In [ ]:
output_folder_left_rectified  = os.path.join(BASE_DIR, 'data_1/multicast_rect/camera1')
output_folder_right_rectified = os.path.join(BASE_DIR, 'data_1/multicast_rect/camera2')

os.makedirs(output_folder_left_rectified, exist_ok=True)
os.makedirs(output_folder_right_rectified, exist_ok=True)

for fname_left, fname_right in zip(list_of_image_files_left, list_of_image_files_right):

    img_left = cv2.imread(os.path.join(image_folder_left, fname_left))
    img_left_rectified = cv2.remap(img_left, map1_left, map2_left, cv2.INTER_LANCZOS4)
    output_path_left = os.path.join(output_folder_left_rectified, fname_left)
    cv2.imwrite(output_path_left, img_left_rectified)

    img_right = cv2.imread(os.path.join(image_folder_right, fname_right))
    img_right_rectified = cv2.remap(img_right, map1_right, map2_right, cv2.INTER_LANCZOS4)
    output_path_right = os.path.join(output_folder_right_rectified, fname_right)
    cv2.imwrite(output_path_right, img_right_rectified)

print("Rectified images and depth maps have been saved.")

# Rectify dataset 2

In [22]:
# pmf_path           = BASE_DIR + '/data_1/ground_truth_pfm_disparity_half'
image_folder_left  = os.path.join(BASE_DIR, 'data_2/multicast/camera1')
image_folder_right = os.path.join(BASE_DIR, 'data_2/multicast/camera2')


# list_of_pfm_files = sorted([pfm for pfm in os.listdir(pmf_path) if pfm.endswith('.pfm')])
list_of_image_files_left = sorted([image for image in os.listdir(image_folder_left) if image.endswith('.jpg')])
list_of_image_files_right = sorted([image for image in os.listdir(image_folder_right) if image.endswith('.jpg')])

In [23]:
output_folder_left_rectified  = os.path.join(BASE_DIR, 'data_2/multicast_rect/camera1')
output_folder_right_rectified = os.path.join(BASE_DIR, 'data_2/multicast_rect/camera2')

os.makedirs(output_folder_left_rectified, exist_ok=True)
os.makedirs(output_folder_right_rectified, exist_ok=True)

for fname_left, fname_right in zip(list_of_image_files_left, list_of_image_files_right):

    img_left = cv2.imread(os.path.join(image_folder_left, fname_left))
    img_left_rectified = cv2.remap(img_left, map1_left, map2_left, cv2.INTER_LANCZOS4)
    output_path_left = os.path.join(output_folder_left_rectified, fname_left)
    cv2.imwrite(output_path_left, img_left_rectified)

    img_right = cv2.imread(os.path.join(image_folder_right, fname_right))
    img_right_rectified = cv2.remap(img_right, map1_right, map2_right, cv2.INTER_LANCZOS4)
    output_path_right = os.path.join(output_folder_right_rectified, fname_right)
    cv2.imwrite(output_path_right, img_right_rectified)

print("Rectified images and depth maps have been saved.")

Rectified images and depth maps have been saved.


# Rectify dataset 3

In [24]:
# pmf_path           = BASE_DIR + '/data_1/ground_truth_pfm_disparity_half'
image_folder_left  = os.path.join(BASE_DIR, 'data_3/multicast/camera1')
image_folder_right = os.path.join(BASE_DIR, 'data_3/multicast/camera2')


# list_of_pfm_files = sorted([pfm for pfm in os.listdir(pmf_path) if pfm.endswith('.pfm')])
list_of_image_files_left = sorted([image for image in os.listdir(image_folder_left) if image.endswith('.jpg')])
list_of_image_files_right = sorted([image for image in os.listdir(image_folder_right) if image.endswith('.jpg')])

In [25]:
output_folder_left_rectified  = os.path.join(BASE_DIR, 'data_3/multicast_rect/camera1')
output_folder_right_rectified = os.path.join(BASE_DIR, 'data_3/multicast_rect/camera2')

os.makedirs(output_folder_left_rectified, exist_ok=True)
os.makedirs(output_folder_right_rectified, exist_ok=True)

for fname_left, fname_right in zip(list_of_image_files_left, list_of_image_files_right):

    img_left = cv2.imread(os.path.join(image_folder_left, fname_left))
    img_left_rectified = cv2.remap(img_left, map1_left, map2_left, cv2.INTER_LANCZOS4)
    output_path_left = os.path.join(output_folder_left_rectified, fname_left)
    cv2.imwrite(output_path_left, img_left_rectified)

    img_right = cv2.imread(os.path.join(image_folder_right, fname_right))
    img_right_rectified = cv2.remap(img_right, map1_right, map2_right, cv2.INTER_LANCZOS4)
    output_path_right = os.path.join(output_folder_right_rectified, fname_right)
    cv2.imwrite(output_path_right, img_right_rectified)

print("Rectified images and depth maps have been saved.")

Rectified images and depth maps have been saved.
